In [ ]:
!pip install torch
!pip install transformers
!pip install datasets
!pip install scikit-learn
!pip install tqdm
!pip install pandas
!pip install plotly
!pip install kaleido
!pip install jupyter
!pip install matplotlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 77.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 10.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires python-multipart, 

In [ ]:
# loading model
from evaluate_detexd_roberta import predict_delicate
from transformers import pipeline

classifier = pipeline("text-classification", model="grammarly/detexd-roberta-base", device=0, batch_size=8)
predict_delicate(classifier, ['hello', 'fuck'])

  0%|          | 0/2 [00:00<?, ?it/s]

[False, True]

## Founta dataset

### Data preparation

In [ ]:
# you can find the original data here: https://zenodo.org/record/3706866#.Y35QVOzMIZ9

import pandas as pd

df = pd.read_table('hatespeech_text_label_vote_RESTRICTED_100K.csv',
                   names=['text', 'label', 'votes'], skiprows=2)
# skipping broken lines

In [ ]:
# the dataset got bigger 80k -> 100k
# there's no information whant changed
# the rows are shuffled and ids removed, so we can not restore original dataset
df.label.value_counts().sort_index()

abusive    27150
hateful     4965
normal     53851
spam       14029
Name: label, dtype: int64

In [ ]:
# removing spam since there are delicate texts there that are not filtered like porn
df = df[df.label != 'spam']

In [ ]:
df.label.value_counts().sort_index()

abusive    27150
hateful     4965
normal     53851
Name: label, dtype: int64

In [ ]:
import datasets
from transformers.pipelines.pt_utils import KeyDataset

preds = predict_delicate(classifier, KeyDataset(datasets.Dataset.from_pandas(df), 'text'))

  0%|          | 0/85966 [00:00<?, ?it/s]

In [ ]:
df['real'] = df.label != 'normal'

In [ ]:
df['pred'] = preds

In [ ]:
df.to_csv('preds.csv', index=False)

### Data analysis

In [ ]:
df = pd.read_csv('preds.csv')

In [ ]:
df.real.mean()

0.3735779261568527

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
founta_results = precision_recall_fscore_support(y_true=df.real, y_pred=df.pred, average='binary')
founta_results

(0.7626132876614572, 0.6655145570605636, 0.7107630401888895, None)

In [ ]:
# FPs
print('\n------\n'.join(df[df.pred & ~df.real].sample(10, random_state=23543).text))

@NuRepublitarian Never forget how Reagan allowed his SCOTUS nominee to be Borked. Never happens to a Dem.
------
@realitycheckind @IndiaNotSecular @pranasutra @learning_pt @mnshzz When do we get our freedom and rights
------
Horny Teacher Was Caught Learning Students How To Have Sex Instead Do Her Job https://t.co/PFXQWA0lD6
------
But...but...who can remove the secret president from the National Security Council? Was it #Putin? So confusing! https://t.co/Eh6hCThqJX
------
12. needs no progress, i will lie by salvia palth- its like porn for ur ears . its beautiful and his hurt voice is just enough to make u nut
------
The vote of #NoConfidence will never succeed as long as it's tabled by the DA. It will only unite the ANC.
------
Cops: Girl, 13, jumps from moving car to escape suspect in kidnap, rape - CBS News https://t.co/EnoEDjnJnS #SmartNews #usaidiots
------
Slave saviours: the men risking their lives to free brick workers in Dagestan https://t.co/zyeEYddyVS
------
RT @fxactivist:

In [ ]:
# FNs
print('\n------\n'.join(df[~df.pred & df.real].sample(10, random_state=23543).text))

#WallStreet Shorting the shit out of oil 55.50 https://t.co/JlCerrkg4n https://t.co/37rkiL8lSB https://t.co/OCbfOJXNOJ
------
That's fucking it I'm going to Cleveland for the weekend nobody talk to me &#128581;&#127998;
------
right now his approval ratings r in the proverbial toilet! someone needs 2 unseat him in the house. i don't mean an… https://t.co/47E0TUsWAP
------
RT @Papapishu: Persona 5 rules even if you're just fucking around in the menus https://t.co/B8pu2laRy1
------
RT @KiiingSauce: you just fucked it up sister &#128525;&#128525;&#128525;&#128557; https://t.co/gJPdfAKCUI
------
RT @prozdkp: when you're mad at video games but controllers are expensive as hell https://t.co/3uBM6t6XTH
------
RT @Stormzy1: #GSAP TOUR - DUBLIN - FUCKING HELL!!!! &#127470;&#127466;❤️&#128170;&#127999; https://t.co/b6uzw8c3Ze
------
@NBA2K_MyTEAM OMG I FUCKING GOT IT https://t.co/tPEldZNFfi
------
RT @mynamesdiana: my fucking heart!!!! https://t.co/dmqE2OnHYb
------
I fucking forgot to cancel it

## Basile dataset

### Data preparation

In [ ]:
# you can find the original data here: http://hatespeech.di.unito.it/hateval.html
# or here: https://github.com/alisonrib17/SemEval2019-Task5/tree/master/English

df = pd.read_table('test_en.tsv')

In [ ]:
df.text.str.contains('bitch', case=False).mean()

0.4834224598930481

In [ ]:
df['pred'] = predict_delicate(classifier, KeyDataset(datasets.Dataset.from_pandas(df), 'text'))
df['real'] = df.HS == 1

  0%|          | 0/2805 [00:00<?, ?it/s]

In [ ]:
df.to_csv('preds_basile.csv', index=False)

### Data analysis

In [ ]:
df = pd.read_csv('preds_basile.csv')

In [ ]:
df.real.mean()

0.4206773618538324

In [ ]:
df.pred.mean()

0.7875222816399287

In [ ]:
basile_results = precision_recall_fscore_support(y_true=df.real, y_pred=df.pred, average='binary')
basile_results

(0.47532820280669985, 0.8898305084745762, 0.6196518146946001, None)

In [ ]:
# FPs
print('\n------\n'.join(df[df.pred & ~df.real].sample(10, random_state=23543).text))

Woody Allen ..uflrghhjj .. hiding behind Hollywood production houses bitch raped his one step daughter and married another
------
im a bitch and have sick sense of humor so no one likes me but i'm just like whatevs boo you whore
------
#FactCheck the #LeftistHypocrisy is staggering #CryingChuck #GoNancyGo #MaxineVirus #Schiffy #BuildTheWall #DeportThemAll #SchumerShutdown never will we forget #NoAmnesty #NiDACA #EndSanctuaryCities #EndVisaLottery #EndChainMigration #StopTheInvasion #AfternoonExpress
------
I am a broke bitch that wants the bts puma shoes and I cannot pay in love
------
It's funny when people say "if someone says they're a boy, they're a boy!" while calling me a "bitch" or a "cunt," referencing my gender.
------
Radical Muslims are granted extra rights. #RadicalIslam #Nosharia #NoMuslimIdeology #BuildThatWall #Trump2020 #MAGA #POTUS45 #Christianity #JesusChrist #2AShallNotBeInfringed
------
Then he shouldn't mind them right ? #BuildTheWall #BuildThatWall #MAGA #AmericaF

In [ ]:
# FNs
print('\n------\n'.join(df[~df.pred & df.real].sample(10, random_state=23543).text))

#IllegalAliens cited in theft of 39 MILLION Social Security Numbers.They especially like the social security numbers of Children!
------
@kissmeyouhoe I love your header bitch https://t.co/ZdcNH3EPup
------
@DiamondandSilk Looks like he doesnt keep his front door open.. but they expect everyone else to.#BuildThatWall #SaturdayVibes
------
@HCKGyanganj @AupolNews Hey, they like fighting so much they can return there - plenty of fighting & violence for them & no one will care.Win win#sendthemback
------
@AMike4761 Wake the fuck up and fight back! #Savethewest #sendthemback
------
@SteveScalise We voted to stop our invasion, not a tax cut.#BuildTheWall #DeportThemAll
------
.@maddow Republicans in #congress dont want to see another Republican #impeached. @DevinNunes & @cathymcmorris on #secret #recording said this all goes away if the Republicans dont keep the #House. #NunesTape shows plot to fire #Rosenstein. #VoteBlueNoMatterWho #SendThemHome
------
The complacent are in every piece of 

In [ ]:
pd.DataFrame([founta_results, basile_results],
             columns=['precision', 'recall', 'f-score', '_'],
             index=['Founta', 'Basile']
            ).iloc[:, :3].style.format('{:.1%}'.format)

,precision,recall,f-score
Founta,76.3%,66.6%,71.1%
Basile,47.5%,89.0%,62.0%


### Restoring precision and recall

In [ ]:
def positive_class_metrics(positive_class_percent, macro_f1, accuracy):
    def f1_macro_fn(tp):
        tn = accuracy - tp
        fp = negative_class_percent - tn
        fn = positive_class_percent - tp
        p1 = tp / (tp + fp + 1e-100)
        r1 = tp / (tp + fn + 1e-100)
        p0 = tn / (tn + fn + 1e-100)
        r0 = tn / (tn + fp + 1e-100)
        f1_fn = p0 * r0 / (p0 + r0 + 1e-100) + p1 * r1 / (p1 + r1 + 1e-100)
        return f1_fn, p1, r1

    negative_class_percent = 1 - positive_class_percent
    n = 1000
    diff, tp_opt = min((abs(f1_macro_fn(tp / n)[0] - macro_f1), tp / n) for tp in range(n + 1))
    _, p1, r1 = f1_macro_fn(tp_opt)
    f1 = 2 / (1 / p1 + 1 / r1)
    print(f'precision: {p1:.1%}')
    print(f'recall: {r1:.1%}')
    print(f'f1: {f1:.1%}')


positive_class_metrics(
    positive_class_percent=0.4206773618538324,
    accuracy=0.65,
    macro_f1=0.651)

precision: 56.1%
recall: 77.3%
f1: 65.0%
